# Le script rag_main charge les embeddings et les chunks depuis embeddings/, puis encode une question utilisateur avec le même modèle d’embeddings pour trouver les chunks les plus similaires via cosine_similarity.
# Il récupère les morceaux de texte pertinents (top_k, ex. 3), les combine en un contexte, et envoie ce contexte avec la question à l’API Gemini pour générer une réponse.
# Cette étape combine la récupération (retrieval) et la génération (augmentation) pour fournir une réponse précise basée sur les données scientifiques extraites des PDFs.

In [5]:
#pip install google-cloud-aiplatform

In [6]:
#pip install einops

In [7]:
#Importations
import os
import numpy as np
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import openai

In [8]:
# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Configuration des modèles et clé API
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  

def get_api_key():
    api_key = os.getenv("GOOGLE_API_KEY")  # Récupérer la clé API depuis .env
    if not api_key:
        api_key = input("Veuillez entrer votre clé API Google : ").strip()
        if not api_key:
            raise ValueError("Aucune clé API fournie. Définissez GOOGLE_API_KEY dans .env ou entrez-la manuellement.")
    return api_key

GOOGLE_API_KEY = get_api_key()

client = openai.OpenAI(
    api_key=GOOGLE_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [9]:
# Cellule 3 : Fonctions de chargement et récupération
def load_embeddings(embedding_dir):
    embedding_path = os.path.join(embedding_dir, "embeddings.npy")
    embeddings = np.load(embedding_path)
    
    chunk_path = os.path.join(embedding_dir, "chunk_mapping.txt")
    chunks = []
    with open(chunk_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            parts = line.strip().split("|||")
            if len(parts) < 2:
                raise ValueError(f"Erreur ligne {i} dans chunk_mapping.txt : moins de 2 parties ({line.strip()})")
            filename = parts[0]
            chunk = "|||".join(parts[1:])
            chunks.append(chunk)
    return embeddings, chunks

def retrieve_relevant_chunks(question, embeddings, chunks, model, top_k=3):
    question_embedding = model.encode(question, convert_to_tensor=False)
    similarities = cosine_similarity([question_embedding], embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [chunks[i] for i in top_indices], similarities[top_indices]

In [10]:
#Génération avec Gemini
def generate_answer_gemini(question, context):
    prompt = f"Question : {question}\nContexte : {context}\nRéponse :"
    response = client.chat.completions.create(
        model="gemini-2.0-flash-exp",
        messages=[
            {"role": "system", "content": "Vous êtes un assistant utile qui répond aux questions basées sur le contexte fourni."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    answer = response.choices[0].message.content
    return answer.split("Réponse :")[1].strip() if "Réponse :" in answer else answer.strip()

In [11]:
# Cellule 5 : Exécution principale
embedding_dir = r"C:\Users\sarto\OneDrive\Bureau\RAG\embeddings"
embeddings, chunks = load_embeddings(embedding_dir)

question = "what are the mechanism behind the fact that execising might improve mental health"
print(f"\nQuestion : {question}")

relevant_chunks, scores = retrieve_relevant_chunks(question, embeddings, chunks, embedding_model)
context = "\n".join(relevant_chunks)
print("Contexte trouvé :", context)

try:
    answer = generate_answer_gemini(question, context)
    print("Réponse générée :", answer)
except Exception as e:
    print(f"Erreur avec l'API Gemini : {e}")


Question : what are the mechanism behind the fact that execising might improve mental health
Contexte trouvé : health ally accepted minimum number of 10 studies needed for conditions. A possible mechanism of action may involve quantitatively estimate the funnel plot asymmetry was elevated levels of brain-derived neurotrophic factor or not present in69(46%) CSRs. beta-endorphins [195]. Strengths of this research are the inclusion of large We found the issue of poor reporting or underreport- number of ‘gold standard’ systematic reviews, robust ing of adherence/withdrawals in over a quarter of CSRs screening, data extractions and critical methodological (25.3%). This is crucial both for improving the accuracy appraisal. Nevertheless, some weaknesses need to be of the estimates at the RCT level as well as maintaining highlighted when interpreting findings of this overview. high levels of physical activity and associated health ben- For instance, some of these CSRs analysed the same pri- e